In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn import preprocessing
import tensorflow as tf

In [11]:
audio_raw = pd.read_csv('Audiobooks_data.csv')
audio_raw

,873,2160,2160.1,10.13,10.13.1,0,8.91,0.1,0.2,0.3,0.4,1
0,611,1404.0,2808,6.66,13.33,1,6.50,0.00,0.0,0,182,1
1,705,324.0,324,10.13,10.13,1,9.00,0.00,0.0,1,334,1
2,391,1620.0,1620,15.31,15.31,0,9.00,0.00,0.0,0,183,1
3,819,432.0,1296,7.11,21.33,1,9.00,0.00,0.0,0,0,1
4,138,2160.0,2160,10.13,10.13,1,9.00,0.00,0.0,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14078,27398,2160.0,2160,7.99,7.99,0,8.91,0.00,0.0,0,54,0
14079,28220,1620.0,1620,5.33,5.33,1,9.00,0.61,0.0,0,4,0
14080,28671,1080.0,1080,6.55,6.55,1,6.00,0.29,0.0,0,29,0
14081,31134,2160.0,2160,6.14,6.14,0,8.91,0.00,0.0,0,0,0


In [12]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

In [30]:
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]
print(len(unscaled_inputs_all))

14084


In [31]:
no_of_ones = int(sum(targets_all))
zero_counter = 0
indices_to_remove = []
for i in range(len(targets_all)):
    if targets_all[i]==0:
        zero_counter += 1
        if zero_counter>no_of_ones:
            indices_to_remove.append(i)
print(len(indices_to_remove))

9610


In [32]:
unscaled_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_all_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)
scaled_inputs = preprocessing.scale(unscaled_equal_priors)
print(unscaled_equal_priors)

[[2.160e+03 2.160e+03 1.013e+01 ... 0.000e+00 0.000e+00 0.000e+00]
 [1.404e+03 2.808e+03 6.660e+00 ... 0.000e+00 0.000e+00 1.820e+02]
 [3.240e+02 3.240e+02 1.013e+01 ... 0.000e+00 1.000e+00 3.340e+02]
 ...
 [2.160e+03 2.160e+03 1.013e+01 ... 2.592e+02 0.000e+00 1.400e+01]
 [2.160e+03 2.160e+03 8.300e+00 ... 2.592e+02 0.000e+00 9.300e+01]
 [2.160e+03 2.160e+03 8.000e+00 ... 2.592e+02 0.000e+00 2.400e+01]]


In [42]:
shufled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shufled_indices)

shufled_inputs = scaled_inputs[shufled_indices]
shufled_targets = targets_all_equal_priors[shufled_indices]

In [44]:
samples_count = shufled_inputs.shape[0]
print(samples_count)
train_samples_count = int(0.8*samples_count)
val_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - val_samples_count

train_inputs = shufled_inputs[:train_samples_count]
train_targets = shufled_targets[:train_samples_count]
print(train_targets.shape[0])
val_inputs = shufled_inputs[train_samples_count:train_samples_count+val_samples_count]
val_targets = shufled_targets[train_samples_count:train_samples_count+val_samples_count]

test_inputs = shufled_inputs[train_samples_count+val_samples_count:]
test_targets = shufled_targets[train_samples_count+val_samples_count:]

print(np.sum(train_targets)/train_targets.shape[0])
print(np.sum(val_targets)/val_targets.shape[0])
print(np.sum(test_targets)/test_targets.shape[0])

4474
3579
0.49958088851634536
0.4966442953020134
0.5066964285714286


In [45]:
np.savez('Audio_train', inputs = train_inputs, targets = train_targets)
np.savez('Audio_test', inputs = test_inputs, targets = test_targets)
np.savez('Audio_val', inputs = val_inputs, targets = val_targets)

In [47]:
train_inputs = train_inputs.astype(np.float)
train_targets = train_targets.astype(np.int)
val_inputs = val_inputs.astype(np.float)
val_targets = val_targets.astype(np.int)
test_inputs = test_inputs.astype(np.float)
test_targets = test_targets.astype(np.int)

In [48]:
train_inputs.shape

(3579, 10)

In [52]:
input_size = 10
output_size = 2
hidden_layers = 50

model = tf.keras.Sequential([
            tf.keras.layers.Dense(hidden_layers, activation='relu'),
            tf.keras.layers.Dense(hidden_layers, activation='relu'),
            tf.keras.layers.Dense(output_size, activation='softmax')
])

In [53]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [60]:
batch_size = 50
max_epochs = 50
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs,
         train_targets,
         batch_size = batch_size,
         epochs = max_epochs,
         callbacks = [early_stopping],
         validation_data=(val_inputs, val_targets))

Epoch 1/50
72/72 [==============================] - 1s 14ms/step - loss: 0.2184 - accuracy: 0.9187 - val_loss: 0.2504 - val_accuracy: 0.9128
Epoch 2/50
72/72 [==============================] - 1s 17ms/step - loss: 0.2145 - accuracy: 0.9195 - val_loss: 0.2475 - val_accuracy: 0.9150
Epoch 3/50
72/72 [==============================] - 1s 13ms/step - loss: 0.2153 - accuracy: 0.9193 - val_loss: 0.2566 - val_accuracy: 0.8926
Epoch 4/50
72/72 [==============================] - 1s 14ms/step - loss: 0.2181 - accuracy: 0.9184 - val_loss: 0.2549 - val_accuracy: 0.9016


In [63]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

14/14 [==============================] - 0s 10ms/step - loss: 0.2320 - accuracy: 0.9107
Test loss: 0.23. Test accuracy: 91.07%
